In [1]:
from tira.third_party_integrations import ir_datasets, persist_and_normalize_run
from tira.rest_api_client import Client
from colbert import Searcher
import pandas as pd
import json
import gzip

tira = Client()
dataset_id = 'reneuir-2024/dl-top-10-docs-20240701-training'
dataset = ir_datasets.load(dataset_id)

In [2]:
topics = {str(i.query_id): i.default_text() for i in dataset.queries_iter()}

In [3]:
plaid_index = tira.get_run_output('reneuir-2024/reneuir-baselines/plaid-x-retrieval', dataset_id) + '/index'
document_ids = json.load(gzip.open(f'{plaid_index}/document_ids.json.gz'))

In [4]:
searcher = Searcher(index=plaid_index)

[Jul 05, 19:43:51] [0] 		 Loading model hltcoe/plaidx-large-eng-tdist-mt5xxl-engeng...
[Jul 05, 19:43:59] #> Loading codec...
[Jul 05, 19:43:59] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Jul 05, 19:43:59] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Jul 05, 19:43:59] #> Loading IVF...
[Jul 05, 19:43:59] #> Loading doclens...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1522.43it/s]

[Jul 05, 19:43:59] #> Loading codes and residuals...



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 188.09it/s]


In [5]:
raw_scores = searcher.search_all(topics, k=1000)


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . who is aziz hashim, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([     0, 250002,      5,   2750,     83,    295,   1418,   1556,  10706,
             2, 250001, 250001, 250001, 250001, 250001, 250001, 250001, 250001,
        250001, 250001, 250001, 250001, 250001, 250001, 250001, 250001, 250001,
        250001, 250001, 250001, 250001, 250001])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:00<00:00, 145.05it/s]


In [6]:
run = []

for qid, ranking in raw_scores.items():
    for didx, _, score in ranking:
        run += [{'qid': qid, 'docno': str(document_ids[didx]), 'score': score}]

run = pd.DataFrame(run)

persist_and_normalize_run(run, 'plaid-x', '.')

The run file is normalized outside the TIRA sandbox, I will store it at ".".
Done. run file is stored under "./run.txt".
